<a href="https://colab.research.google.com/github/tthandi/magic_formula_project/blob/main/CompanyOverviewLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
alpha_vantage_company_overview.py — COMPANY-OVERVIEW

• Fetches Alpha Vantage OVERVIEW (company metadata) for US stocks (active + delisted).
• Partitioned by yyyymm (month of fetch). Writes with WRITE_APPEND, then de-dups.
• Modes:
    full     → all symbols
    gapfill  → symbols missing for current yyyymm
    backfill → symbols already present for current yyyymm (refresh), then de-dup
    auto     → full if current month empty else gapfill
• Premium friendly: CALLS_PER_MIN=75, CONCURRENCY=50, async + rate-limited.
"""

# ───────────────────────────── CONFIG ───────────────────────────────
PROJECT   = ""
DATASET   = ""
TABLE     = "company_overview"

import os as _os
API_KEY   = '' # REQUIRED

CALLS_PER_MIN = 75
CONCURRENCY   = 50
FLUSH_ROWS    = 5_000
CHECKFILE     = "av_done_symbols_co.txt"
FAILFILE      = "av_failed_symbols_co.txt"
PART_START, PART_END = 198001, 204001  # yyyymm

# ───────────────────────────── IMPORTS ──────────────────────────────
import io, csv, json, time, asyncio, tempfile, aiohttp, nest_asyncio, argparse, datetime
from collections import defaultdict
from typing import Dict, List, Set, Tuple
import pandas as pd
import requests
import tqdm

from aiolimiter import AsyncLimiter
from google.cloud import bigquery
from google.api_core import exceptions as gex
from google.auth.exceptions import DefaultCredentialsError, RefreshError
from google.oauth2 import service_account

nest_asyncio.apply()

# ───────────────────────── Auth-robust BigQuery client ──────────────
def get_bq_client() -> bigquery.Client:
    # 1) Colab/Notebook user auth
    try:
        from google.colab import auth as colab_auth  # type: ignore
        colab_auth.authenticate_user()
        return bigquery.Client(project=PROJECT)
    except Exception:
        pass
    # 2) Service account
    sa_path = _os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
    if sa_path and _os.path.exists(sa_path):
        creds = service_account.Credentials.from_service_account_file(sa_path)
        return bigquery.Client(project=PROJECT, credentials=creds)
    # 3) ADC (gcloud)
    try:
        return bigquery.Client(project=PROJECT)
    except (DefaultCredentialsError, RefreshError) as e:
        raise RuntimeError(
            "No Google Cloud credentials found. Use Colab auth, a service account "
            "via GOOGLE_APPLICATION_CREDENTIALS, or `gcloud auth application-default login`."
        ) from e

bq = get_bq_client()

# ───────────────────── Alpha Vantage async helpers ──────────────────
BASE_URL     = "https://www.alphavantage.co/query"
RATE_LIMITER = AsyncLimiter(CALLS_PER_MIN, 60)

async def av_request(session: aiohttp.ClientSession, params: dict) -> dict | None:
    """Generic AV GET with rate limiting & error handling."""
    async with RATE_LIMITER:
        try:
            async with session.get(BASE_URL, params=params, timeout=45) as resp:
                if resp.status != 200:
                    return None
                return await resp.json(content_type=None)
        except (aiohttp.ClientError, asyncio.TimeoutError):
            return None

async def listing(session: aiohttp.ClientSession, state: str, max_retries: int = 6) -> List[str]:
    """Robust LISTING_STATUS CSV → list of stock symbols."""
    params = {"function": "LISTING_STATUS", "state": state, "datatype": "csv", "apikey": API_KEY}
    backoff = 2
    for attempt in range(1, max_retries + 1):
        async with RATE_LIMITER:
            try:
                async with session.get(BASE_URL, params=params, timeout=45) as resp:
                    status = resp.status
                    txt = await resp.text()
            except (aiohttp.ClientError, asyncio.TimeoutError):
                tqdm.tqdm.write(f"⚠️ LISTING_STATUS {state}: network error (try {attempt})")
                await asyncio.sleep(backoff); backoff *= 2
                continue

        if txt.lstrip().startswith("{"):
            # Quota/info JSON even with 200 OK
            try:
                j = json.loads(txt)
            except json.JSONDecodeError:
                j = {}
            msg = j.get("Note") or j.get("Information") or j.get("Error Message") or f"HTTP {status}"
            tqdm.tqdm.write(f"⚠️ LISTING_STATUS {state}: {msg} (try {attempt})")
            await asyncio.sleep(backoff); backoff = min(backoff * 2, 30)
            continue

        rdr = csv.DictReader(io.StringIO(txt))
        syms = [r["symbol"] for r in rdr if r.get("assetType") == "Stock" and r.get("symbol")]
        if syms:
            return syms

        tqdm.tqdm.write(f"⚠️ LISTING_STATUS {state}: empty CSV (try {attempt})")
        await asyncio.sleep(backoff); backoff *= 2

    tqdm.tqdm.write(f"❗ LISTING_STATUS {state}: giving up after {max_retries} attempts")
    return []

# ─────────────────────── Overview fetch & shaping ───────────────────
TODAY        = datetime.date.today()
FETCH_YYYYMM = TODAY.year * 100 + TODAY.month

def coerce_numeric(col: pd.Series) -> pd.Series:
    col = col.replace({"None": pd.NA, "null": pd.NA, "": pd.NA})
    return pd.to_numeric(col, errors="coerce")

async def fetch_overview(session: aiohttp.ClientSession, sym: str) -> pd.DataFrame | None:
    """Return a one-row DataFrame with selected overview fields for *sym*."""
    data = await av_request(session, {"function": "OVERVIEW", "symbol": sym, "apikey": API_KEY})
    # Alpha Vantage returns {} for unknown symbols
    if not data or "Symbol" not in data:
        return None

    keep_map = {
        "Symbol": "symbol",
        "MarketCapitalization": "marketCap",
        "PERatio": "peRatio",
        "Sector": "sector",
        "Industry": "industry",
        "Country": "country",
    }
    row = {new: data.get(old) for old, new in keep_map.items()}
    row["yyyymm"] = FETCH_YYYYMM
    df = pd.DataFrame([row])
    # numeric casts
    for c in ["marketCap", "peRatio"]:
        if c in df.columns:
            df[c] = coerce_numeric(df[c])
    return df

# ───────────────────────── BigQuery boot-strapping ──────────────────
# Dataset
try:
    bq.get_dataset(f"{PROJECT}.{DATASET}")
except gex.NotFound:
    bq.create_dataset(bigquery.Dataset(f"{PROJECT}.{DATASET}"))

table_ref = f"{PROJECT}.{DATASET}.{TABLE}"

SCHEMA = [
    bigquery.SchemaField("symbol",    "STRING"),
    bigquery.SchemaField("marketCap", "FLOAT64"),
    bigquery.SchemaField("peRatio",   "FLOAT64"),
    bigquery.SchemaField("sector",    "STRING"),
    bigquery.SchemaField("industry",  "STRING"),
    bigquery.SchemaField("country",   "STRING"),
    bigquery.SchemaField("yyyymm",    "INT64"),
]

def ensure_table():
    try:
        tbl = bq.get_table(table_ref)
        existing = {f.name for f in tbl.schema}
        need = [f for f in SCHEMA if f.name not in existing]
        if need:
            tbl.schema += need
            bq.update_table(tbl, ["schema"])
            tqdm.tqdm.write(f"🆕 added missing columns → {', '.join(f.name for f in need)}")
    except gex.NotFound:
        tbl = bigquery.Table(table_ref, schema=SCHEMA)
        tbl.range_partitioning = bigquery.RangePartitioning(
            field="yyyymm", range_=bigquery.PartitionRange(PART_START, PART_END, 1)
        )
        tbl.clustering_fields = ["symbol"]
        bq.create_table(tbl)
        tqdm.tqdm.write(f"➡️  created table {table_ref}")

ensure_table()
CSV_COLS = [f.name for f in bq.get_table(table_ref).schema]  # keep order consistent

# ───────────────────────── Load & Dedupe helpers ────────────────────
def load_partition(df_part: pd.DataFrame, yyyymm: int, retries: int = 4):
    """Append df_part to TABLE$yyyymm via gzip CSV."""
    if df_part is None or df_part.empty:
        return
    df_part = df_part.reindex(columns=CSV_COLS)
    fd, tmp = tempfile.mkstemp(suffix=".csv.gz"); _os.close(fd)
    df_part.to_csv(tmp, index=False, compression="gzip", date_format="%Y-%m-%d")

    dest = f"{table_ref}${yyyymm}"  # explicitly target this partition
    cfg = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        write_disposition="WRITE_APPEND",
    )
    backoff = 5
    for attempt in range(1, retries + 1):
        try:
            with open(tmp, "rb") as fh:
                job = bq.load_table_from_file(fh, dest, job_config=cfg, rewind=True)
            tqdm.tqdm.write(f"⬆️  load {dest} rows {len(df_part)} (try {attempt})")
            job.result(timeout=300)
            tqdm.tqdm.write(f"✅ partition {yyyymm} rows {len(df_part)}")
            break
        except (gex.GoogleAPICallError, gex.RetryError, TimeoutError) as e:
            tqdm.tqdm.write(f"⚠️  {dest} {e} — backoff {backoff}s")
            time.sleep(backoff); backoff = min(backoff * 2, 60)
    _os.remove(tmp)

def dedupe_month(yyyymm: int):
    """Keep one row per (symbol, yyyymm)."""
    sql = f"""
    CREATE OR REPLACE TEMP TABLE tmp AS
    SELECT * EXCEPT(rn) FROM (
      SELECT t.*,
             ROW_NUMBER() OVER (
               PARTITION BY symbol, yyyymm
               ORDER BY symbol
             ) rn
      FROM `{table_ref}` t
      WHERE yyyymm = @p
    )
    WHERE rn = 1;

    DELETE FROM `{table_ref}` WHERE yyyymm = @p;
    INSERT INTO `{table_ref}` SELECT * FROM tmp;
    """
    job = bq.query(
        sql,
        job_config=bigquery.QueryJobConfig(
            query_parameters=[bigquery.ScalarQueryParameter("p", "INT64", yyyymm)]
        ),
    )
    job.result()
    tqdm.tqdm.write(f"🧹 dedupe complete for {yyyymm}")

# ───────────────────────── Checkpoint / symbols ─────────────────────
def read_checkpoint() -> Set[str]:
    return set(open(CHECKFILE).read().splitlines()) if _os.path.exists(CHECKFILE) else set()

def add_checkpoint(sym: str):
    with open(CHECKFILE, "a") as f:
        f.write(sym + "\n")

def add_failure(sym: str):
    with open(FAILFILE, "a") as f:
        f.write(sym + "\n")

def symbols_loaded_for_month(yyyymm: int) -> Set[str]:
    q = f"SELECT DISTINCT symbol FROM `{table_ref}` WHERE yyyymm = @p"
    rows = bq.query(q, job_config=bigquery.QueryJobConfig(
        query_parameters=[bigquery.ScalarQueryParameter("p", "INT64", yyyymm)]
    )).result()
    return {r.symbol for r in rows}

# ───────────────────────── Mode selection helper ────────────────────
def decide_mode(requested: str, existing_for_month: Set[str], universe: List[str]) -> str:
    if requested != "auto":
        return requested
    if not existing_for_month:
        tqdm.tqdm.write("🧭 Auto: current month empty → FULL")
        return "full"
    tqdm.tqdm.write("🧭 Auto: current month has data → GAP-FILL")
    return "gapfill"

# ───────────────────────── Main async entry-point ───────────────────
async def run(mode: str):
    if not API_KEY:
        raise SystemExit("❌ Set ALPHAVANTAGE_API_KEY and retry.")

    pending: List[pd.DataFrame] = []
    sem       = asyncio.Semaphore(CONCURRENCY)
    connector = aiohttp.TCPConnector(limit=CONCURRENCY)

    async with aiohttp.ClientSession(connector=connector,
                                     timeout=aiohttp.ClientTimeout(total=90)) as session:
        active, delisted = await asyncio.gather(
            listing(session, "active"),
            listing(session, "delisted"),
        )
        tqdm.tqdm.write(f"ℹ️ LISTING_STATUS → active: {len(active):,}, delisted: {len(delisted):,}")
        universe = sorted(set(active + delisted))
        if not universe:
            tqdm.tqdm.write("❗ LISTING_STATUS returned 0; aborting (premium key expected to work).")
            return

        existing_month = symbols_loaded_for_month(FETCH_YYYYMM)
        mode = decide_mode(mode, existing_month, universe)

        if mode == "full":
            todo = universe
            desc = "FULL crawl (overview)"
        elif mode == "gapfill":
            todo = [s for s in universe if s not in existing_month]
            desc = "GAP-FILL (missing for current month)"
        elif mode == "backfill":
            todo = [s for s in universe if s in existing_month]
            desc = "BACKFILL (already present this month)"
        else:
            raise SystemExit(f"Unknown mode: {mode}")

        # (Optional) also factor in historical success checkpoint if you want resume semantics
        done = read_checkpoint()
        # Only skip by checkpoint for full/backfill; for gapfill we already filtered by BQ presence
        if mode in ("full", "backfill"):
            todo = [s for s in todo if s not in done]

        tqdm.tqdm.write(f"Mode: {desc} | universe {len(universe):,} | "
                        f"in-month {len(existing_month):,} | to fetch {len(todo):,}")

        pbar = tqdm.tqdm(total=len(todo), desc="Symbols processed", dynamic_ncols=True)

        async def worker(sym: str):
            async with sem:
                df = await fetch_overview(session, sym)
                if df is not None and not df.empty:
                    pending.append(df)
                    # flush on threshold (single month partition)
                    if sum(len(d) for d in pending) >= FLUSH_ROWS:
                        batch = pd.concat(pending); pending.clear()
                        await asyncio.to_thread(load_partition, batch, FETCH_YYYYMM)
                    add_checkpoint(sym)  # success-only
                else:
                    add_failure(sym)
                pbar.update(1)

        await asyncio.gather(*(asyncio.create_task(worker(s)) for s in todo))
        pbar.close()

    # Final flush
    if pending:
        await asyncio.to_thread(load_partition, pd.concat(pending), FETCH_YYYYMM)

    # De-dup the current month
    await asyncio.to_thread(dedupe_month, FETCH_YYYYMM)

# ────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode", choices=["auto", "full", "gapfill", "backfill"],
                        default="auto",
                        help="auto=full if current month empty else gap-fill")
    args, _unknown = parser.parse_known_args()  # Jupyter-friendly
    asyncio.run(run(args.mode))
